In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import cv2
from random import shuffle
from tqdm import tqdm
import os

In [ ]:
train = '../input/train'
test = '../input/test'

In [ ]:
img_size = 50
lr = 0.001

In [ ]:
def label_image(img):
    word_label = img.split('.')[-3]
    if word_label == 'dog':
        return [1, 0]
    elif word_label == 'cat':
        return [0, 1]

In [ ]:
def create_training_data():
    training_data = []
    for img in os.listdir(train):
        label = label_image(img)
        path = os.path.join(train, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (img_size, img_size))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    return training_data

In [ ]:
def process_data():
    testing_data = []
    for img in os.listdir(test):
        path = os.path.join(test,img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(img_size,img_size))
        testing_data.append([np.array(img),img_num])
        return testing_data
        

In [ ]:
train_data = create_training_data()

In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, img_size, img_size, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

In [ ]:
train_d = train_data[: -500]
test_d = train_data[-500:]

In [ ]:
X = np.array([i[0] for i in train_d]).reshape(-1, img_size, img_size, 1)
Y = np.array([i[1]for i in train_d])

test_x =  np.array([i[0] for i in test_d]).reshape(-1, img_size, img_size, 1)
test_y = np.array([i[1]for i in test_d])

In [ ]:
model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True)